# Dissertation

## Research into the techniques and methods to achieve state of the art accuracy in flower species identification


In [204]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [205]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path

In [206]:
print(tf.__version__)

2.5.0


In [207]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [208]:
if tf.test.gpu_device_name(): 
  
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [209]:
#makes all outputs be in float format rather than exponentials
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [223]:
#Set Parameters
IMG_SIZE = 250 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'FlowerClassifierTrial.model'.format(int(time.time()))
log_dir=f'logs\\{MODEL_NAME}'
TENSORBOARD = TensorBoard(log_dir)

In [224]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Load Directory

In [225]:
# ###Directory for MacBook
# mac_URL = '/Volumes/T7/Uni/Github/thesis/flowerDataset/Flower'
# mac_labels = '/Volumes/T7/Uni/Github/thesis/flowerDataset/imagelabels.mat'
# mac_dataSplit = '/Volumes/T7/Uni/Github/thesis/flowerDataset/setid.mat'
# mac_loaded_images = '/Volumes/T7/Uni/loaded_images.npy'
# mac_image_train='/Volumes/T7/Uni/image_train.npy'
# mac_label_train = '/Volumes/T7/Uni/label_train.npy'
# mac_image_test = '/Volumes/T7/Uni/image_test.npy'
# mac_label_test = '/Volumes/T7/Uni/label_test.npy'
# mac_image_val = '/Volumes/T7/Uni/image_val.npy'
# mac_label_val = '/Volumes/T7/Uni/label_val.npy'

# ###Directory for PC
# pc_URL = 'E:/Github/thesis/flowerDataset/Flower'
# pc_labels = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
# pc_dataSplit = 'E:/Github/thesis/flowerDataset/setid.mat'
# pc_loaded_images = 'E:/Dissertation/data/loaded_images.npy'
# pc_image_train = 'E:/Dissertation/data/image_train.npy'
# pc_label_train = 'E:/Dissertation/data/label_train.npy'
# pc_image_test = 'E:/Dissertation/data/image_test.npy'
# pc_label_test = 'E:/Dissertation/data/label_test.npy'
# pc_image_val = 'E:/Dissertation/data/image_val.npy'
# pc_label_val = 'E:/Dissertation/data/label_val.npy'

In [226]:
def get_directories():
    if os.path.exists(mac_URL) and os.path.exists(mac_labels) and os.path.exists(
            mac_dataSplit) and os.path.exists(mac_loaded_images):
        return mac_URL, mac_labels, mac_dataSplit, mac_loaded_images, mac_image_train, mac_label_train, mac_image_test, mac_label_test,mac_image_val, mac_label_val 
    else:
        return pc_URL, pc_labels, pc_dataSplit, pc_loaded_images, pc_image_train, pc_label_train, pc_image_test, pc_label_test, pc_image_val, pc_label_val

In [227]:
#URL_dir, labels_dir, dataSplit_dir, loaded_images_dir, image_train_dir, label_train_dir, image_test_dir, label_test_dir, image_val_dir, label_val_dir = get_directories()

In [228]:
URL_dir = 'E:/Github/thesis/flowerDataset/Flower'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
dataSplit_dir = 'E:/Github/thesis/flowerDataset/setid.mat'
loaded_images_dir = 'E:/Dissertation/data/loaded_images.npy'
image_train_dir ='E:/Dissertation/data/image_train.npy'
label_train_dir = 'E:/Dissertation/data/label_train.npy'
image_test_dir = 'E:/Dissertation/data/image_test.npy'
label_test_dir = 'E:/Dissertation/data/label_test.npy'
image_val_dir = 'E:/Dissertation/data/image_val.npy'
label_val_dir = 'E:/Dissertation/data/label_val.npy'

In [229]:
print(labels_dir)

E:/Github/thesis/flowerDataset/imagelabels.mat


### Load DataSet and Labels

In [230]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)

In [231]:
df = pd.DataFrame(data_labels)

In [232]:
labels = df[1][3][0] #loads the labels that is stored in dataframe format

In [233]:
labels

array([77, 77, 77, ..., 62, 62, 62], dtype=uint8)

In [234]:
len(labels)

8189

In [235]:
UNIQUE_LABELS = np.unique(labels)
UNIQUE_LABELS

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102], dtype=uint8)

In [236]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        #img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
    return images

In [237]:
#No need to run this cell if image already loaded.
load_images_from_folder(URL_dir)

[array([[[ 95, 119, 125],
         [ 88, 112, 118],
         [ 76, 102, 108],
         ...,
         [  7,  20,  18],
         [ 12,  25,  23],
         [ 16,  29,  27]],
 
        [[102, 124, 130],
         [ 92, 116, 122],
         [ 78, 106, 111],
         ...,
         [  4,  17,  15],
         [  8,  21,  19],
         [ 12,  25,  23]],
 
        [[110, 132, 137],
         [ 97, 123, 127],
         [ 79, 111, 114],
         ...,
         [  0,  13,  11],
         [  4,  17,  15],
         [  7,  20,  18]],
 
        ...,
 
        [[114, 147, 137],
         [103, 142, 131],
         [ 89, 133, 123],
         ...,
         [ 34,  51,  70],
         [ 36,  53,  72],
         [ 36,  53,  72]],
 
        [[114, 154, 146],
         [ 99, 143, 134],
         [ 89, 137, 125],
         ...,
         [ 31,  48,  67],
         [ 35,  52,  71],
         [ 34,  51,  70]],
 
        [[115, 156, 149],
         [101, 146, 137],
         [ 91, 140, 128],
         ...,
         [ 31,  48,  67],
  

In [238]:
#if images are already loaded, just load the file here rather than running function again.
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [239]:
#shows the shape of the image_data, the number of images, the dimensions and number of colour channels
image_data.shape

(8189, 250, 250, 3)

In [240]:
image_data[0].shape

(250, 250, 3)

In [241]:
#combine image with labels
def randomised_dataset(image_data, labels):
    np.random.seed(8)
    indices = np.arange(image_data.shape[0])
    print(indices)
    np.random.shuffle(indices)
    image_data = image_data[indices]
    labels = labels[indices]
    return image_data, labels

In [242]:
image_data, labels = randomised_dataset(image_data, labels)

[   0    1    2 ... 8186 8187 8188]


In [243]:
print(image_data.shape)

(8189, 250, 250, 3)


In [244]:
# # Label Encoder

mlb = LabelBinarizer()
converted_labels = np.array(mlb.fit_transform(labels))

In [245]:
converted_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [246]:
def convert_label(labels):
    new_labels = []
    for i in range(len(labels)):
        new_labels.insert(i, [labels[i]])
    return np.array(new_labels)

In [247]:
#converted_labels = convert_label(labels)

In [248]:
#converted_labels.shape

In [249]:
len(converted_labels)

8189

In [250]:
converted_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Training and Testing Split

In [251]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation and testing data
def split_data(image_data, converted_labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, converted_labels, test_size=1 - train_ratio, random_state=42)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)



In [252]:
split_data(image_data,converted_labels)

image_train (6141, 250, 250, 3)
label_train (6141, 102)
image_test (820, 250, 250, 3)
label_test (820, 102)
image_val (1228, 250, 250, 3)
label_val (1228, 102)


In [253]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

In [254]:
image_test.shape

(820, 250, 250, 3)

In [255]:
label_val[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Feature Scaling

In [256]:
#converts the images to the range of 0 - 1.0 
image_train = image_train/255.0
image_test = image_test/255.0
image_val = image_val/255.0

In [257]:
image_train.shape

(6141, 250, 250, 3)

In [258]:
label_train.shape

(6141, 102)

In [259]:
label_test.shape

(820, 102)

In [260]:
# number of classes/ flowers
NUM_CLASSES = 102
NUM_CLASSES

102

# Custom CNN model

In [261]:
batch_size = 32
epochs = 100
verbose= 2
act = 'relu' #sigmoid tanh
dropout_val = 0.4

In [262]:
optimizer = tf.keras.optimizers.Adam(0.01)

In [263]:
def create_cnn_model(act):
    # tf.reset_default_graph()
    model = Sequential()
    input_shape = (IMG_SIZE, IMG_SIZE, 3)
    
    # INPUT LAYER
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation(act))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    # HIDDEN LAYER 1
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))

#     #HIDDEN LAYER 2
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))
    
#     # HIDDEN LAYER 3
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))
    
    # HIDDEN LAYER 4
    model.add(Conv2D(256, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))

    # Fully Connected
    model.add(Flatten()) # converts the 3D feature maps to 1D feature vectors
#     model.add(Dense(128))
#     model.add(Activation(act))
#     model.add(Dropout(dropout_val)) # reduces overfitting

    # OUTPUT LAYER
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [264]:
model = create_cnn_model(act)

In [265]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 248, 248, 32)      896       
_________________________________________________________________
activation_24 (Activation)   (None, 248, 248, 32)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 246, 246, 32)      9248      
_________________________________________________________________
activation_25 (Activation)   (None, 246, 246, 32)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 123, 123, 32)      0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 123, 123, 32)      128       
_________________________________________________________________
dropout_16 (Dropout)         (None, 123, 123, 32)     

In [266]:
#history = model.fit(image_train, label_train, batch_size=64, epochs=100, validation_data=(image_val, label_val), verbose=1, callbacks = [tensorboard])

In [267]:
#saving model
#model.save(f'{MODEL_PATH}')

In [268]:
#loading model
#model =  tf.keras.models.load_model(f'{MODEL_PATH}')
#model = tf.keras.models.load_model(f'E:/GoogleSync/Masters/Dissertation/models/24-Jun-2021') 

# Evaluation

In [269]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [270]:
EXPERIMENT_NAME = "custom-model-tests"
RUN_NAME = "img_size_250"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_param("activation function", act)
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        
        
        
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.keras.log_model(model, RUN_NAME)        

2021/08/19 10:03:28 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Epoch 1/100


192/192 - 17s - loss: 10.6662 - accuracy: 0.1443 - val_loss: 23.0368 - val_accuracy: 0.0285
Epoch 2/100
192/192 - 9s - loss: 8.6495 - accuracy: 0.3355 - val_loss: 18.5781 - val_accuracy: 0.0774
Epoch 3/100
192/192 - 9s - loss: 4.2399 - accuracy: 0.5742 - val_loss: 10.6644 - val_accuracy: 0.2809
Epoch 4/100
192/192 - 9s - loss: 2.0326 - accuracy: 0.7240 - val_loss: 10.0023 - val_accuracy: 0.2964
Epoch 5/100
192/192 - 9s - loss: 1.1268 - accuracy: 0.8248 - val_loss: 8.1759 - val_accuracy: 0.3787
Epoch 6/100
192/192 - 9s - loss: 0.6901 - accuracy: 0.8738 - val_loss: 9.1061 - val_accuracy: 0.3795
Epoch 7/100
192/192 - 9s - loss: 0.4438 - accuracy: 0.9150 - val_loss: 8.8102 - val_accuracy: 0.4047
Epoch 8/100
192/192 - 8s - loss: 0.3431 - accuracy: 0.9254 - val_loss: 8.7188 - val_accuracy: 0.4186
Epoch 9/100
192/192 - 8s - loss: 0.4304 - accuracy: 0.9204 - val_loss: 10.1587 - val_accuracy: 0.3795
Epoch 10/100
192/192 - 8s - loss: 0.4008 - accuracy: 0.9253 - val_loss: 9.6368 - val_accuracy: 0

192/192 - 9s - loss: 0.0380 - accuracy: 0.9933 - val_loss: 10.0907 - val_accuracy: 0.4577
Epoch 82/100
192/192 - 9s - loss: 0.0368 - accuracy: 0.9948 - val_loss: 9.6751 - val_accuracy: 0.4381
Epoch 83/100
192/192 - 9s - loss: 0.0297 - accuracy: 0.9963 - val_loss: 9.4674 - val_accuracy: 0.4300
Epoch 84/100
192/192 - 9s - loss: 0.0338 - accuracy: 0.9945 - val_loss: 10.1496 - val_accuracy: 0.4397
Epoch 85/100
192/192 - 9s - loss: 0.0587 - accuracy: 0.9909 - val_loss: 10.5074 - val_accuracy: 0.3893
Epoch 86/100
192/192 - 9s - loss: 0.0326 - accuracy: 0.9946 - val_loss: 10.4656 - val_accuracy: 0.4300
Epoch 87/100
192/192 - 9s - loss: 0.0421 - accuracy: 0.9919 - val_loss: 9.7116 - val_accuracy: 0.4609
Epoch 88/100
192/192 - 9s - loss: 0.0567 - accuracy: 0.9897 - val_loss: 9.3761 - val_accuracy: 0.4471
Epoch 89/100
192/192 - 9s - loss: 0.0392 - accuracy: 0.9927 - val_loss: 10.7623 - val_accuracy: 0.4267
Epoch 90/100
192/192 - 9s - loss: 0.0463 - accuracy: 0.9917 - val_loss: 10.3314 - val_accu

INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmp70cjzgvh\model\data\model\assets


26/26 - 1s - loss: 9.3090 - accuracy: 0.4500
INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpdmrr3has\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpdmrr3has\model\data\model\assets


In [271]:
import winsound

winsound.Beep(1000, 1000)

In [ ]:
#function that returns loss value & metrics values
model.evaluate(image_test, label_test)

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir=logs

In [ ]:
def prediction_accuracy(image_test, model):
    #going through each of the test images
    count = 0
    for i in range(len(image_test)): #going through all test images
        img = image_test[i] #load in the image
        
        img = img.reshape(-1, 100, 100, 3) #get it in the right shape for model.predict
        
        prediction = model.predict(img).flatten() #using model predict to get what it thinks is the answer
        
        # prediction.astype(np.int) #converts predictions to integers
        
        pred_index = np.argmax(prediction) # grabs the INDEX of the best prediction
        
        max_pred = max(prediction) # grabs the SCORE of best prediction
        # print(prediction)
        # print(pred_index)
        
        test_label_value = label_test[i][pred_index] # gets the actual test label at the same index
        # print(test_label_value)
        if test_label_value == 1:
            #print("CORRECT PREDICTION")
            count += 1
            
#         # FIND THE INDEX WHERE THE VALUE = 1
#         for j in range(len(predicted_label)):
#             if predicted_label[j] == 1 : #find the point when it finds 1 
#                 if label_test[i][j] == 1: #check if at this exact point is also 1  whic means correct prediction
#                     count+=1 #increase number of correct count by 1
#                     print(count)
#                     break
    print(f"num correct - {count}")
    accuracy = (count/len(label_test))*100
    per_symbol = '%'
    print(f'accuracy is {accuracy} {per_symbol}')        

In [ ]:
image_test.shape

In [ ]:
prediction_accuracy(image_test, model) 